# SPARC-4 products
***

This notebook shows an example for viewing and accessing the SPARC4 data products.

* Eder Martioli, LNA 24 Nov 2023

In [ ]:
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sparc4.product_plots as s4plt
import sparc4.pipeline_lib as s4pipelib

### Set example file paths
Here we assign a string variable to each file path used in this notebook

In [ ]:
db_example = '/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/sparc4acs1/20230606/20230606_sparc4acs1_db.fits'

phot_stack_example = '/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/sparc4acs1/20230606/20230606_s4c1_Gain1_1_Conventional_2_ltt4816_stack.fits'

phot_lightcurve_example ='/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/sparc4acs1/20230606/20230606_s4c1_ltt4816_lc.fits'

polar_proc_example = '/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/sparc4acs1/20230604/20230604_s4c1_000471_stdpol_proc.fits'

polar_stack_example = '/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/sparc4acs1/20230604/20230604_s4c1_Hilt652_POLAR_L2_stack.fits'

polar_example = '/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/sparc4acs1/20230604/20230604_s4c1_000471_stdpol_l2_polar.fits'

polar_time_series_example = '/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/sparc4acs1/20230604/20230604_s4c1_Hilt652_POLAR_L2_ts.fits'

# DB product example
***
In this simple example, we just read the database created by the `sparc4-pipelin` and print it as a pandas table. It's an easy way to quickly access the log of observations.

In [ ]:
db = fits.open(db_example)
pd.DataFrame(db[1].data)

# PHOT products example
***
In this example we show how to work with the photometric products of SPARC4.

First we show how to load a science FITS product using `astropy` and print information about the data structure and contents in this file.

In [ ]:
hdul = fits.open(phot_stack_example)
hdul.info()

Then, we print the primary header, which is typical of a SPARC4 processed frame:

In [ ]:
print(repr(hdul[0].header))

Now print one of the catalogs to see its contents

In [ ]:
catalog = 'CATALOG_PHOT_AP012'
pd.DataFrame(hdul[catalog].data)

Now let's plot the image data using matplotlib:

In [ ]:
img_data = hdul[0].data
plt.imshow(img_data, vmin=np.percentile(img_data, 0.5), vmax=np.percentile(img_data, 99.5))

Now we also plot the same image but using the `sparc4.product_plots` library from `sparc4-pipeline`

In [ ]:
s4plt.plot_sci_frame(phot_stack_example, cat_ext=catalog, nstars=5, use_sky_coords=True)

Now we load the light curve FITS product and check its contents:

In [ ]:
lc = fits.open(phot_lightcurve_example)
lc.info()

In [ ]:
pd.DataFrame(lc["CATALOG_PHOT_AP010"].data)

Below we select the source with index number 3 assigned as the main target to be analyzed. Then we select three other stars with indices 0, 1, and 2, as comparisons for differential photometry. All indices are displayed in the image above.  

After selecting the stars we use the function `s4plt.plot_light_curve` to produce plots of the lightcurve data.

In [ ]:
target=3
comps=[0,1,2]

s4plt.plot_light_curve(phot_lightcurve_example, target=target, comps=comps, nsig=10,
                     plot_coords=True, plot_rawmags=True, plot_sum=True,
                     plot_comps=True, catalog_name=catalog)

# POLAR products example
***
In this example we show how to work with the polarimetric products of SPARC4

We start by reading the science image FITS file and showing its information. 

In [ ]:
hdul = fits.open(polar_stack_example)
hdul.info()

Then we plot the science frame using the `sparc4.product_plots` library

In [ ]:
s4plt.plot_sci_polar_frame(polar_stack_example)

Now we open a polarimetry FITS product and print its basic information

In [ ]:
polar = fits.open(polar_example)
polar.info()

Below we print one of the catalog extensions

In [ ]:
pd.DataFrame(polar["POLARIMETRY_AP012"].data)

Then we can print all column names to check the data available in each catalog

In [ ]:
pd.DataFrame(polar["POLARIMETRY_AP012"].data).columns

Finally, we can select the source of interest (in this case index=0) and then we get the polarimetry results using the function `get_polarimetry_results` from `sparc4.pipeline_lib` library:

In [ ]:
source_index = 0

pol_results = s4pipelib.get_polarimetry_results(polar_example,
                                                source_index=source_index,
                                                min_aperture=4,
                                                max_aperture=25,
                                                plot=True,
                                                verbose=True)

Below we print all variables names for the measurements available in the polar product

In [ ]:
for key in pol_results.keys():
    print(key, ":", pol_results[key])

And below we show a simple example how to get one of these measurements out. For instance, we're getting the total polarization and the polarization angle. 

In [ ]:
p = pol_results["P"]*100
pa = pol_results["THETA"]
print("Polarization: {} %  Angle of polarization: {} deg".format(p,pa) )

Finally, we show below an example of a polarimetric time series product. 

In [ ]:
polar_ts = fits.open(polar_time_series_example)
pd.DataFrame(polar_ts[1].data)

In [ ]:
pd.DataFrame(polar_ts[1].data).columns

In [ ]:
s4plt.plot_polar_time_series(polar_time_series_example, 
                             target=0, 
                             comps=[1,2,3], 
                             plot_total_polarization=True, 
                             plot_polarization_angle=True)